In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from implementations import *
from train import *

%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = '../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
tX, tX_test = replace_data(tX, replace=-999, by=np.NaN), replace_data(tX_test, replace= -999, by=np.NaN)

y = replace_data(y, -1, 0)

In [5]:
# initialize the range of hyperparameters
degrees = np.arange(2, 6)
lambdas = np.logspace(-4, 0, 25)

# initialize the number of k-fold
k_fold = 6

In [6]:
best_weights, best_degree = train_models(y, tX, degrees, lambdas, k_fold)

In [7]:
best_degree

[5, 3, 5, 4, 5, 5]

## Generate predictions and save ouput in csv format for submission:

In [8]:
# get the indices of the subsets in the training and the test datasets
indices_train_group = group_indices(tX)
indices_test_group = group_indices(tX_test)

In [9]:
# initialize the prediction array
y_pred = np.zeros(tX_test.shape[0])

y_pred = np.zeros(tX_test.shape[0])
for i, indice_test_group in enumerate(indices_test_group):
    y_pred_subset = predict_labels(best_weights[i], build_poly(drop_na_columns(tX_test[indice_test_group]), best_degree[i]))
    y_pred[indice_test_group] = y_pred_subset
    
    
y_pred

In [10]:
### TEST ###

for i, indice_test_group in enumerate(indices_test_group):
    
    # for standardizing the test subset, we need the data of both train and test subsets
    tx_subset = tX[indices_train_group[i]]
    tx_test_subset = tX_test[indice_test_group]
    
    # get the standardized test subset
    _, standardized_tx_test_subset = preprocess_data(tx_subset, tx_test_subset)
    
    # predict the labels
    y_pred_subset = predict_labels(best_weights[i], build_poly(standardized_tx_test_subset, best_degree[i]))
    y_pred[indice_test_group] = y_pred_subset
    
    
y_pred

array([-1., -1., -1., ...,  1., -1., -1.])

In [11]:
OUTPUT_PATH = '../data/sample-submission_reg_log.csv'
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)